In [ ]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType

In [ ]:
#Warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/nba'
#Create Spark Session
builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [6]:
#Read CSV to DataFrame
file_location = 'hdfs://hdfs-nn:9000/projeto/triplos/bronze/nba_players'
#header=true e inferschema=true
df = spark\
    .read\
    .format("csv")\
    .option('header','true')\
    .option("inferSchema" , "true")\
    .option("delimiter",",")\
    .load(file_location)

df.printSchema()
df.show()

root
 |-- _c0: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- G: integer (nullable = true)
 |-- GS: integer (nullable = true)
 |-- MP: integer (nullable = true)
 |-- PER: double (nullable = true)
 |-- TS%: double (nullable = true)
 |-- 3PAr: double (nullable = true)
 |-- FTr: double (nullable = true)
 |-- ORB%: double (nullable = true)
 |-- DRB%: double (nullable = true)
 |-- TRB%: double (nullable = true)
 |-- AST%: double (nullable = true)
 |-- STL%: double (nullable = true)
 |-- BLK%: double (nullable = true)
 |-- TOV%: double (nullable = true)
 |-- USG%: double (nullable = true)
 |-- blanl: string (nullable = true)
 |-- OWS: double (nullable = true)
 |-- DWS: double (nullable = true)
 |-- WS: double (nullable = true)
 |-- WS/48: double (nullable = true)
 |-- blank2: string (nullable = true)
 |-- OBPM: double (nullable = 

21/11/29 23:28:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Year, Player, Pos, Age, Tm, G, GS, MP, PER, TS%, 3PAr, FTr, ORB%, DRB%, TRB%, AST%, STL%, BLK%, TOV%, USG%, blanl, OWS, DWS, WS, WS/48, blank2, OBPM, DBPM, BPM, VORP, FG, FGA, FG%, 3P, 3PA, 3P%, 2P, 2PA, 2P%, eFG%, FT, FTA, FT%, ORB, DRB, TRB, AST, STL, BLK, TOV, PF, PTS
 Schema: _c0, Year, Player, Pos, Age, Tm, G, GS, MP, PER, TS%, 3PAr, FTr, ORB%, DRB%, TRB%, AST%, STL%, BLK%, TOV%, USG%, blanl, OWS, DWS, WS, WS/48, blank2, OBPM, DBPM, BPM, VORP, FG, FGA, FG%, 3P, 3PA, 3P%, 2P, 2PA, 2P%, eFG%, FT, FTA, FT%, ORB, DRB, TRB, AST, STL, BLK, TOV, PF, PTS
Expected: _c0 but found: 
CSV file: hdfs://hdfs-nn:9000/projeto/triplos/bronze/nba_players/nba_players.csv


In [46]:
#Create Deltatable in DataBase NBA empty_ STRING,
spark.sql(
    """
    CREATE EXTERNAL TABLE nba.bz_players (
        id INT, player_name STRING, position STRING, age INT, team STRING, games INT, games_started INT,
        minutes_played INT, player_eficiency_rating DOUBLE, true_shooting_percentage DOUBLE, 3point_attempt_rate DOUBLE,
        free_throw_rate DOUBLE, offensive_rebound_percentage DOUBLE, defensive_rebound_percentage DOUBLE,
        total_rebound_percentage DOUBLE, assist_percentage DOUBLE, steal_percentage DOUBLE, block_percentage DOUBLE,
        turnover_percentage DOUBLE, usage_percentage DOUBLE,  blank_ STRING, offensive_win_shares DOUBLE,
        deffensive_win_shares DOUBLE, win_shares DOUBLE, win_shares_48min DOUBLE, blank2_ STRING, offensivebox_plusminus DOUBLE,
        deffensivebox_plusminus DOUBLE, box_plusminus DOUBLE, value_over_replacement DOUBLE, field_goals INT,
        field_goals_attempts INT, field_goal_percentage DOUBLE, 3p_field_goals INT,
        3p_field_goals_attempts INT, 3p_field_goals_percentage DOUBLE, 2p_field_goals INT,
        2p_field_goals_attempts INT, 2p_field_goals_percentage DOUBLE, effective_field_gold_percentage DOUBLE,
        free_thows INT, free_thows_attempt INT, free_thows_percentage DOUBLE, offensive_rebound INT,
        defensive_rebound INT, total_rebound INT, assists INT, steals INT, blocks INT, turnover INT,
        personal_fouls INT, points INT
    )
    USING DELTA
    PARTITIONED BY(
    year INT
    )
    TBLPROPERTIES (
    'skip.header.line.count'='1'
    )
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/bz_players'
    """
)


21/11/30 00:07:32 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `nba`.`bz_players` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [47]:
#show databases [mostrar todas as bases de dados]
#show tables from nba [mostrar as tabelas da base de dados ...]
#drop table if exists nba.players_exp [apagar a tabela ... da base de dados ...]
#select *  from nba.players_exp [mostrar os dados de ... tabela]

spark.sql(
"""
select *  from nba.bz_players
""").show()

+---+-----------+--------+---+----+-----+-------------+--------------+-----------------------+------------------------+-------------------+---------------+----------------------------+----------------------------+------------------------+-----------------+----------------+----------------+-------------------+----------------+------+--------------------+---------------------+----------+----------------+-------+----------------------+-----------------------+-------------+----------------------+-----------+--------------------+---------------------+--------------+-----------------------+-------------------------+--------------+-----------------------+-------------------------+-------------------------------+----------+------------------+---------------------+-----------------+-----------------+-------------+-------+------+------+--------+--------------+------+----+
| id|player_name|position|age|team|games|games_started|minutes_played|player_eficiency_rating|true_shooting_percentage|3point

In [49]:
#Write DataFrame to Deltatable
df \
    .select("*") \
    .write \
    .mode("overwrite") \
    .partitionBy("Year") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save("hdfs://hdfs-nn:9000/projeto/nba/bz_players")

21/11/30 00:09:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Year, Player, Pos, Age, Tm, G, GS, MP, PER, TS%, 3PAr, FTr, ORB%, DRB%, TRB%, AST%, STL%, BLK%, TOV%, USG%, blanl, OWS, DWS, WS, WS/48, blank2, OBPM, DBPM, BPM, VORP, FG, FGA, FG%, 3P, 3PA, 3P%, 2P, 2PA, 2P%, eFG%, FT, FTA, FT%, ORB, DRB, TRB, AST, STL, BLK, TOV, PF, PTS
 Schema: _c0, Year, Player, Pos, Age, Tm, G, GS, MP, PER, TS%, 3PAr, FTr, ORB%, DRB%, TRB%, AST%, STL%, BLK%, TOV%, USG%, blanl, OWS, DWS, WS, WS/48, blank2, OBPM, DBPM, BPM, VORP, FG, FGA, FG%, 3P, 3PA, 3P%, 2P, 2PA, 2P%, eFG%, FT, FTA, FT%, ORB, DRB, TRB, AST, STL, BLK, TOV, PF, PTS
Expected: _c0 but found: 
CSV file: hdfs://hdfs-nn:9000/projeto/triplos/bronze/nba_players/nba_players.csv


In [50]:
spark.sql(
"""
select * from nba.bz_players
where Year == 1980
"""
).show()

+----+----+--------------------+---+---+---+---+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+----+-----+----+---+----+------+------+----+----+----+----+---+----+-----+---+---+-----+---+----+-----+-----+---+---+-----+---+---+---+---+---+---+---+---+----+
| _c0|Year|              Player|Pos|Age| Tm|  G|  GS|  MP| PER|  TS%| 3PAr|  FTr|ORB%|DRB%|TRB%|AST%|STL%|BLK%|TOV%|USG%|blanl| OWS|DWS|  WS| WS/48|blank2|OBPM|DBPM| BPM|VORP| FG| FGA|  FG%| 3P|3PA|  3P%| 2P| 2PA|  2P%| eFG%| FT|FTA|  FT%|ORB|DRB|TRB|AST|STL|BLK|TOV| PF| PTS|
+----+----+--------------------+---+---+---+---+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+----+-----+----+---+----+------+------+----+----+----+----+---+----+-----+---+---+-----+---+----+-----+-----+---+---+-----+---+---+---+---+---+---+---+---+----+
|5727|1980|Kareem Abdul-Jabbar*|  C| 32|LAL| 82|null|3143|25.3|0.639|0.001|0.344| 7.2|22.2|15.4|16.5| 1.2| 4.6|15.7|24.1| null| 9.5|5.3|14.8| 0.227|  null| 4.0| 2.7| 6.7